# General coordinate transformation scripts

For changing projections/coordinate reference systems for display on dashboard

In [1]:
import geopandas as gpd
import os
import pandas as pd
from pathlib import Path

# Set up paths
data_dir = Path('../data/raw/testing_coord_transformation')
output_dir = Path('../data/processed/testing_coord_transformations')
output_dir.mkdir(parents=True, exist_ok=True)

print(f"Input directory: {data_dir}")
print(f"Output directory: {output_dir}")

Input directory: ../data/raw/testing_coord_transformation
Output directory: ../data/processed/testing_coord_transformations


In [2]:
# Lists to track files
files_checked = []
files_modified = []
files_unchanged = []

Check what files are present in the directory to ensure correct files are being processed

In [3]:
# Find all geojson files in the data directory
geojson_files = list(data_dir.glob('*.geojson'))

print(f"Found {len(geojson_files)} GeoJSON files")


Found 2 GeoJSON files


Check output directory before going ahead, just to see that you're not overwriting files uninentionally

In [5]:
# find all geojson in output directory

print(f"Found {len(list(output_dir.glob('*.geojson')))} GeoJSON files in output dir")

print(list(output_dir.glob('*.geojson')))

Found 0 GeoJSON files in output dir
[]


If you're happy with the output of the above cells, can run this:

In [7]:
for file_path in geojson_files:
    try:
        # Read the GeoJSON file
        gdf = gpd.read_file(file_path)
        files_checked.append(file_path.name)
        
        # Get the current CRS
        current_crs = gdf.crs
        print(f"\nProcessing: {file_path.name}")
        print(f"Current CRS: {current_crs}")
        
        # Check if CRS is not WGS84 (EPSG:4326)
        if current_crs is None:
            print(f"Warning: No CRS defined for {file_path.name}")
            # Save as unchanged but with warning
            output_file = output_dir / f"{file_path.stem}_nocrs{file_path.suffix}"
            gdf.to_file(output_file, driver='GeoJSON')
            files_unchanged.append(file_path.name)
            
        elif current_crs.to_epsg() != 4326:
            # Convert to WGS84
            gdf_transformed = gdf.to_crs('EPSG:4326')
            output_file = output_dir / f"{file_path.stem}_newcrs{file_path.suffix}"
            gdf_transformed.to_file(output_file, driver='GeoJSON')
            files_modified.append(file_path.name)
            print(f"Converted from {current_crs.to_epsg()} to EPSG:4326")
            
        else:
            # Already in WGS84, copy unchanged
            output_file = output_dir / f"{file_path.stem}_unmodifiedcrs{file_path.suffix}"
            gdf.to_file(output_file, driver='GeoJSON')
            files_unchanged.append(file_path.name)
            print("Already in WGS84 (EPSG:4326)")
            
    except Exception as e:
        print(f"Error processing {file_path.name}: {e}")
        files_checked.append(f"{file_path.name} (ERROR)")

print("Done")



Processing: OSM_highway_crossings-stops-others.geojson
Current CRS: EPSG:27700
Converted from 27700 to EPSG:4326

Processing: OSM_highway_bus_stop.geojson
Current CRS: EPSG:27700
Converted from 27700 to EPSG:4326
Done


In [8]:
# Print summary
print("\n" + "="*50)
print("PROCESSING SUMMARY")
print("="*50)
print(f"Total files checked: {len(files_checked)}")
print(f"Files modified (converted to WGS84): {len(files_modified)}")
print(f"Files unchanged (already WGS84 or no CRS): {len(files_unchanged)}")

print(f"\nFiles checked: {files_checked}")
print(f"\nFiles modified: {files_modified}")
print(f"\nFiles unchanged: {files_unchanged}")



PROCESSING SUMMARY
Total files checked: 4
Files modified (converted to WGS84): 4
Files unchanged (already WGS84 or no CRS): 0

Files checked: ['OSM_highway_crossings-stops-others.geojson', 'OSM_highway_bus_stop.geojson', 'OSM_highway_crossings-stops-others.geojson', 'OSM_highway_bus_stop.geojson']

Files modified: ['OSM_highway_crossings-stops-others.geojson', 'OSM_highway_bus_stop.geojson', 'OSM_highway_crossings-stops-others.geojson', 'OSM_highway_bus_stop.geojson']

Files unchanged: []


In [ ]:
# If desired, can create a summary dataframe
# summary_data = {
#     'filename': files_checked,
#     'status': (['modified'] * len(files_modified) + 
#               ['unchanged'] * len(files_unchanged) + 
#               ['error'] * (len(files_checked) - len(files_modified) - len(files_unchanged)))
# }

# summary_df = pd.DataFrame(summary_data)
# summary_file = output_dir / 'coordinate_transformation_summary.csv'
# summary_df.to_csv(summary_file, index=False)
# print(f"\nSummary saved to: {summary_file}")